In [ ]:
# keeps column where condition holds
# import scipy
# import numpy as np
# from scipy import linalg, matrix
# def null(A, eps=1e-15):
#     u, s, vh = scipy.linalg.svd(A)
# #     v1 = v[:,s>=EPSILON]
#     null_mask = (s <= eps)
#     null_space = scipy.compress(null_mask, vh, axis=0)
#     return (null_space)
# A  = np.matrix([[   0           , 0 ,           0 ,           0 ,           0  ,          0],
#           [ 0    ,      0.5 ,           0,         -0.5,            0 ,-3.06162e-17],
#           [ 0  ,          0     ,       0 ,           0   ,         0   ,         0],
#            [0         ,-0.5            ,0        ,  0.5            ,0 ,-3.06162e-17],
#            [0     ,       0      ,      0   ,         0    ,        0    ,        0],
#            [0 ,-3.06162e-17       ,     0 ,-3.06162e-17     ,       0         ,   1]])

# null(A)

In [ ]:
# EPSILON = 2.220446049250313e-16
# np.matrix.round(A,9)
# #print(A)
# u,s,vt = np.linalg.svd(A)
# #print("sigma",s)
# # print("v matrix",vt)
# vt1 = np.vectorize(lambda x: 0.0 
#                   if -0.01 <= x <= 0.01
#                   else x)(vt) 
# print("v matrix after vectorizing",vt1)
# print(s>=EPSILON)
# v1 = vt1[:,s >= EPSILON]   
# print("v1", v1)
# v1 = np.vectorize(lambda x: 0.0 
#                   if -0.01 <= x <= 0.01
#                   else x)(v1) 
# print("v1 after vectorizing",v1)
# # Filter out all rows that are associated with almost-zero singular values
# # v1 = np.vectorize(lambda x: 0.0 
# #                   if -0.01 <= x <= 0.01
# #                   else x)(v1) #
# # Threshold almost-zero values to zero
# # unidentifiable = (v1 == 0.0).all(axis=ROW_AXIS)


In [5]:
import scipy
from scipy import random, linalg
import numpy as np
import timeit
import time
# import matplotlib.pyplot as plt
from matplotlib import pyplot
from functools import partial

In [10]:
#generate a random PSD matrix 
n = 6 
A = random.rand(n,n)
L = np.dot(A,A.transpose())
#     print("*****Random positive semi-define matrix is*****")
#L[0,1] = 0
#L[1,0] = 0
#L[2,1] = 0
#L[1,2] = 0
#L[2,0] = 0
# print(L)
# np.linalg.cholesky(L)
#validate if L input matrix is a PositiveSemiDefinite
eigvalues, eigvectors = np.linalg.eigh(L.round(3))
# print("***eigvalues of L matrix***")
# print(eigvalues)

#perform **LDL decompostion** np.cholesky
#lu, d, perm = scipy.linalg.ldl(L,lower=1)
lu, d, perm = scipy.linalg.ldl(L)
#     print("****Lower triangular matrix *****")
lu = lu[perm, :]
#     print(lu)
# print(lu[perm,:])
# print("permutation",perm)
no_of_segments = 7;
nTests = 1000;
#perform rank one update ***sentana***
EZ = ([1.0,1.0,2.0,2.0,0.5,2.0])
aD = np.diag(d) 
aD_new = np.zeros(6)
aL_new = np.zeros((6, 6))
aL = lu
beta = np.zeros(6)
D = 1.0
alpha = 1.0 / D 
w = np.copy(EZ)
for p in range(7):
    for q in range(nTests):
        start = time.time();
        for j in range(0,6):
            if not np.allclose(w[j], 0.0):
                if not np.allclose(aD[j], 0.0):
                    p = w[j]
                    aD_new[j] = aD[j] + alpha * (p ** 2.0)
                    b =  alpha / aD_new[j]
                    beta[j] = p * b
                    alpha = b * aD[j]
                    for r in range(j, 6):
                        w[r] = w[r] - (p * aL[r, j])
                        aL_new[r, j] = aL[r, j] + beta[j] * w[r]
                else:
                    p = w[j]
                    aD_new[j] = alpha * (p ** 2.0)
                    beta[j] = 1.0 / p
                    for r in range(j, 6):
                        aL_new[r,j] = beta[j] * w[r]
                    for i in range(j+1,6):
                        aD_new[j] = aD[i]
        #                 for i in range(j+1,6):
                        for r in range(i,6):
                            aL_new[r,i] = aL[r,i]
                    break
            else: 
                aD_new[j] = aD[j]
                alpha = alpha
                for r in range(j,6):
                    w[r] = w[r]
                    aL_new[r,j] = aL[r,j]
            end = time.time();
print("execution time", start-end)


# print("****Alpha after rank one update****")
# print(alpha)
# print("****Input aD****")
# print(aD)
# print("****new calculated values of aD by rank one update******")
# print(aD_new)
# print("*****new calculated values of aL by rank one update******")
# print(aL_new)

execution time -0.00019407272338867188


In [3]:
#now perform validation
#perform L to L' rank one update **L' = L + alpha(zz_t)
def ldl_decomposition(N):
    z = EZ
    # print(EZ)
    alpha = 1.0
    outer_product_z = np.outer(z,z)
    for i in range(6):
        L_new = L + alpha * (outer_product_z)

# print("*****L' after rank one update ****")
# print(L_new)

#LDL decomposition on the L'
# lu, d, perm = scipy.linalg.ldl(L_new)
# print("aL*********")
# lu = lu[perm, :]
# print(lu)
# print("****new aD elements*****")
# print(np.diag(d))
def svd_decomposition(N):
    L_bar = L_new
    U,S,Vt = np.linalg.svd(L_bar)

In [4]:
def plot_timecomplexity(fn, nInc, nTests):
    """
    Run timer and plot time complexity
    """
    x = []
    y = []
# for j in range(6):
    for i in range(nInc):
        N = i
        testNTimer = timeit.Timer(partial(fn, N))
        t = testNTimer.timeit(number=nTests)
        x.append(i)
#         print(x)
        y.append(t)
    p1 = pyplot.plot(x, y, '-')

def main():
    print('Analyzing Algorithms...')
    plot_timecomplexity(rank_one_update, 10, 100)
#   plot_timecomplexity(ldl_decomposition, 10, 10000)
#     plot_timecomplexity(svd_decomposition, 10, 10000)
#     plot_timecomplexity(rank_one_update, 10, 10000)
    #plotTC(fshuffle, 10, 1000, 1000, 10)
#     plot_timecomplexity(fsort, 10, 1000, 10, 10)

    # enable this in case you want to set y axis limits
    #pyplot.ylim((-0.1, 0.5))
    
    # show plot
    pyplot.show()

# call main
if __name__ == '__main__':
    main()


Analyzing Algorithms...


NameError: name 'time' is not defined

In [ ]:
# import numpy as np
# EZ = [1,2,3,5,0.5,2]
# aD_new = np.zeros(6)
# aL_new = np.zeros((6, 6))
# aL = lu
# # print(aL[2:1])
# beta = np.zeros(6)
# alpha = 1.0 / D
# w = EZ  #list
# for j in range(0, 6):
#     if not np.allclose(w[j], 0.0):
#         if not np.allclose(aD[j], 0.0):
#             p = w[j]
#             aD_new[j] = aD[j] + (alpha * (p**2))
# #             print("aD",aD[j])
# #             print("alpha",alpha)
# #             print("p",p)
#             beta[j] = (p * alpha) / aD_new[j]
#             alpha = (aD[j] * alpha) / aD_new[j]
#             for r in range(j+1, 6):
#                 w[r] = w[r] - (p * aL[r, j])
#                 aL_new[r, j] = aL[r, j] + (beta[j] * w[r])
#         else:
#             p = w[j]
#             aD_new[j] = alpha * (p ** 2)
#             beta[j] = 1.0 / p
#             for r in range(j + 1, 6):
#                 aL_new[r,j] = beta[j] * w[r]
#             for i in range(j+1,6):
#                 aD_new[j] = aD[i]
#                 for i in range(j+1,6):
#                     for r in range(i+1,6):
#                         aL_new[r,i] = aL[r,i]
#             break
#     else:       
#         aD_new[j] = aD[j]
#         alpha = alpha
#         for r in range(j+1,6):
#             w[r] = w[r]
#             aL_new[r,j] = aL[r,j] 
# print("alpha",alpha)
# print("AD",aD)
# print("new aD",aD_new)
# print("new aL",aL_new.round(3))

In [ ]:
#validation
# L = np.matrix([[ 0.  , 0.,   0.,   0.,   0. ,  0. ],
#  [ 0.,  -0.5,  0. ,  0.5 , 0. ,  0. ],
#  [ 0.   0.   0.   0.   0.   0. ],
#  [ 0.   0.5  0.  -0.5  0.   0. ],
#  [ 0.   0.   0.   0.   0.   0. ],
#  [ 0.   0.   0.   0.   0.  -1. ]])
# EZ = [1,2,3,5,0.5,2]
# w = EZ
# alpha = 1.0 / D
# w_transpose = np.transpose(w)
# print("w transpose",w_transpose)
# outer_product_z = np.outer(w,w_transpose)
# # print("outer ",outer_product_z)
# L_new = L + alpha * outer_product_z
# print("L matrix rank one", L_new)

# #LDL decomposition
# al, ad, permutation_matrix = scipy.linalg.ldl(L_new,lower=1)
# print("alnew",al)
# print("ad_new",ad.diagonal())


# # #comparing two matrices
# # np.allclose(aL_new,al)

In [ ]:
# # importing the required modules 
import timeit 

setup = '''import scipy
from scipy import random, linalg
import numpy as np
import time
import matplotlib.pyplot as plt
'''
# L = np.array([[1.91985358, 0.87478191, 1.6551661,  1.04511843 ,1.29330246 ,1.50710214],
#  [0.87478191 ,0.48305739 ,0.76457468 ,0.61967858 ,0.49485202 ,0.80090485],
#  [1.6551661 , 0.76457468 ,1.96700866 ,1.32055592 ,1.07233062 ,1.81523297],
#  [1.04511843, 0.61967858, 1.32055592, 1.37976047, 0.70207552, 1.49714724],
#  [1.29330246, 0.49485202 ,1.07233062 ,0.70207552 ,1.43348287 ,1.02876048],
#  [1.50710214 ,0.80090485, 1.81523297 ,1.49714724, 1.02876048 ,1.88340838]])
# z = ([1.0,1.0,2.0,2.0,0.5,2.0])
# alpha = 1.0
# outer_product_z = np.outer(z,z)
# for i in range(7):
#     L_new = L + alpha * (outer_product_z)
# '''

times = timeit.Timer(setup=setup).repeat(10,1000)
print("time ldl {}".format(min(times)))

In [3]:
import scipy
scipy.__version__

'1.1.0'